In [1]:

import pandas as pd

# Carregue seu arquivo CSV
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/arquivomesclado.csv')

# Preencha os valores NaN utilizando o método 'fillna' com 'ffill' e 'bfill'
df_filled = df.fillna(method='ffill').fillna(method='bfill')

# Para as linhas onde 'ffill' e 'bfill' não funcionam (como no começo e no final do DataFrame),
# você pode optar por preencher com a média global da coluna, se isso fizer sentido para seus dados
df_filled = df_filled.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x)

# Salve o DataFrame preenchido em um novo arquivo CSV
df_filled.to_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv', index=False)



In [3]:
import pandas as pd

# Carregar os conjuntos de dados
df_dolar = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv')
df_ibovespa = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibovespa.csv')
df_price_coffee = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/price_coffee.csv')
df_selic = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/selic.csv')
df_weather = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/weather.csv')

# Conversão das colunas de data para datetime
dfs = [df_dolar, df_ibovespa, df_price_coffee, df_selic, df_weather]
for df in dfs:
    df['Date'] = pd.to_datetime(df['Date'])

# Sincronizar as datas com o conjunto price_coffee
dfs_filtered = [df[df['Date'].isin(df_price_coffee['Date'])] for df in dfs]

# Mesclar os conjuntos de dados
df_merged = df_price_coffee
for df in dfs_filtered:
    df_merged = df_merged.merge(df, on='Date', how='left')

# Salvar o DataFrame mesclado em um novo arquivo CSV
df_merged.to_csv(('C:/Users/milen/OneDrive/Documentos/TCC/Bases/arquivomesclado.csv'), index=False)


In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats

# Carregar o conjunto de dados
file_path = 'C:/Users/milen/OneDrive/Documentos/TCC/new/final.csv'  # Substitua pelo caminho correto do seu arquivo
data = pd.read_csv(file_path)

# Convertendo 'Data' para formato datetime e 'Preco_Dolar' para float
data['Data'] = pd.to_datetime(data['Data'])
data['Preco_Dolar'] = data['Preco_Dolar'].str.replace(',', '.').astype(float)

# Tratamento de Outliers
z_scores = np.abs(stats.zscore(data['Preco_Dolar']))
threshold = 3
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar'].where(z_scores < threshold, np.nan)

# Interpolando valores NaN resultantes do tratamento de outliers
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar_No_Outliers'].interpolate()

# Preparando os dados para o modelo de Random Forest
X = data[['Preco_Dolar_No_Outliers']]  # Usando apenas o preço do dólar sem outliers
y = data['Preco_Dolar']  # Valor real do preço do dólar

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=365, shuffle=False)

# Treinamento do modelo Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)

# Realizando a previsão para os últimos 5 valores
rf_forecast = rf_model.predict(X_test)

# Avaliando o modelo
mse_rf = mean_squared_error(y_test, rf_forecast)
r2_rf = r2_score(y_test, rf_forecast)

# Exibindo os resultados
print("MSE:", mse_rf)
print("R2 Score:", r2_rf)

# Preparando os dados de resultado para exibição
result_data_rf = pd.DataFrame({
    'Date': data['Data'].iloc[-365:],
    'Real Value': y_test.values,
    'Forecast Value': rf_forecast
})

print(result_data_rf.reset_index(drop=True))


MSE: 0.006944562328766365
R2 Score: 0.9999934676028719
          Date  Real Value  Forecast Value
0   2022-05-30      268.04        268.2440
1   2022-05-31      267.96        268.2440
2   2022-06-01      275.51        275.5040
3   2022-06-02      277.12        276.9530
4   2022-06-03      270.35        270.3637
..         ...         ...             ...
360 2023-11-06      178.19        178.1976
361 2023-11-07      178.62        178.6228
362 2023-11-08      181.48        181.4768
363 2023-11-09      181.93        181.9296
364 2023-11-10      180.91        180.8922

[365 rows x 3 columns]


In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats

# Carregar o conjunto de dados
file_path = 'C:/Users/milen/OneDrive/Documentos/TCC/new/final.csv'  # Substitua pelo caminho correto do seu arquivo
data = pd.read_csv(file_path)

# Convertendo 'Data' para formato datetime e 'Preco_Dolar' para float
data['Data'] = pd.to_datetime(data['Data'])
data['Preco_Dolar'] = data['Preco_Dolar'].str.replace(',', '.').astype(float)

# Tratamento de Outliers
z_scores = np.abs(stats.zscore(data['Preco_Dolar']))
threshold = 3
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar'].where(z_scores < threshold, np.nan)

# Interpolando valores NaN resultantes do tratamento de outliers
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar_No_Outliers'].interpolate()

# Separando os dados em conjuntos de treinamento e teste
limite_data = pd.to_datetime("2022-01-01")
treinamento = data[data['Data'] < limite_data]
teste = data[data['Data'] >= limite_data]

X_train = treinamento[['Preco_Dolar_No_Outliers']]
y_train = treinamento['Preco_Dolar']
X_test = teste[['Preco_Dolar_No_Outliers']]
y_test = teste['Preco_Dolar']

# Treinamento do modelo Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)

# Realizando a previsão para os dados de teste
rf_forecast = rf_model.predict(X_test)

# Avaliando o modelo
mse_rf = mean_squared_error(y_test, rf_forecast)
r2_rf = r2_score(y_test, rf_forecast)

# Exibindo os resultados
print("MSE:", mse_rf)
print("R2 Score:", r2_rf)

# Preparando os dados de resultado para exibição
result_data_rf = pd.DataFrame({
    'Date': teste['Data'],
    'Real Value': y_test.values,
    'Forecast Value': rf_forecast
})

print(result_data_rf.reset_index(drop=True))


MSE: 0.010763983562230437
R2 Score: 0.9999926500769082
          Date  Real Value  Forecast Value
0   2022-01-03      249.88        249.7722
1   2022-01-04      255.79        255.7616
2   2022-01-05      256.64        256.7849
3   2022-01-06      257.95        257.8179
4   2022-01-07      265.18        265.2447
..         ...         ...             ...
461 2023-11-06      178.19        178.1976
462 2023-11-07      178.62        178.6228
463 2023-11-08      181.48        181.4768
464 2023-11-09      181.93        181.9296
465 2023-11-10      180.91        180.8922

[466 rows x 3 columns]


In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats

# Carregar o conjunto de dados
file_path = 'C:/Users/milen/OneDrive/Documentos/TCC/new/final.csv' # Substitua pelo caminho correto do seu arquivo
data = pd.read_csv(file_path)

# Convertendo 'Data' para formato datetime e 'Preco_Dolar' para float
# Corrigindo o problema de conversão para float na coluna 'Preco_Dolar'
data['Data'] = pd.to_datetime(data['Data'])
data['Preco_Dolar'] = data['Preco_Dolar'].str.replace('.', '').str.replace(',', '.').astype(float)

# Tratamento de Outliers
z_scores = np.abs(stats.zscore(data['Preco_Dolar']))
threshold = 3
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar'].where(z_scores < threshold, np.nan)

# Interpolando valores NaN resultantes do tratamento de outliers
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar_No_Outliers'].interpolate()

# Separando os dados em conjuntos de treinamento e teste
limite_data = pd.to_datetime("2022-01-01")
treinamento = data[data['Data'] < limite_data]
teste = data[data['Data'] >= limite_data]

X_train = treinamento[['Preco_Dolar_No_Outliers']]
y_train = treinamento['Preco_Dolar']
X_test = teste[['Preco_Dolar_No_Outliers']]
y_test = teste['Preco_Dolar']

# Treinamento do modelo Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)

# Realizando a previsão para os dados de teste
rf_forecast = rf_model.predict(X_test)

# Avaliando o modelo
mse_rf = mean_squared_error(y_test, rf_forecast)
r2_rf = r2_score(y_test, rf_forecast)

# Exibindo os resultados
print("MSE:", mse_rf)
print("R2 Score:", r2_rf)

# Preparando os dados de resultado para exibição
result_data_rf = pd.DataFrame({
    'Date': teste['Data'],
    'Real Value': y_test.values,
    'Forecast Value': rf_forecast
})

print(result_data_rf.reset_index(drop=True))


C:\Users\milen\AppData\Local\Temp\ipykernel_25484\426840804.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Preco_Real'] = data['Preco_Real'].str.replace('.', '').str.replace(',', '.').astype(float)


MSE: 4661.2448051827005
R2 Score: 0.8929573705598122
          Date  Real Value  Forecast Value
0   2022-01-03     1414.57       1434.2192
1   2022-01-04     1454.67       1434.2192
2   2022-01-05     1453.35       1434.2192
3   2022-01-06     1466.69       1434.2192
4   2022-01-07     1493.73       1434.2192
..         ...         ...             ...
461 2023-11-06      871.52        871.8496
462 2023-11-07      869.72        869.5415
463 2023-11-08      890.14        883.8229
464 2023-11-09      896.75        894.6933
465 2023-11-10      888.46        882.7719

[466 rows x 3 columns]


In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats

# Carregar o conjunto de dados
file_path = 'C:/Users/milen/OneDrive/Documentos/TCC/new/final.csv' # Substitua pelo caminho correto do seu arquivo
data = pd.read_csv(file_path)

# Convertendo 'Data' para formato datetime e 'Preco_Dolar' para float
# Corrigindo o problema de conversão para float na coluna 'Preco_Dolar'
data['Data'] = pd.to_datetime(data['Data'])
data['Preco_Dolar'] = data['Preco_Dolar'].str.replace('.', '').str.replace(',', '.').astype(float)

# Tratamento de Outliers
z_scores = np.abs(stats.zscore(data['Preco_Dolar']))
threshold = 3
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar'].where(z_scores < threshold, np.nan)

# Interpolando valores NaN resultantes do tratamento de outliers
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar_No_Outliers'].interpolate()

# Separando os dados em conjuntos de treinamento e teste
limite_data = pd.to_datetime("2022-01-01")
treinamento = data[data['Data'] < limite_data]
teste = data[data['Data'] >= limite_data]

X_train = treinamento[['Preco_Dolar_No_Outliers']]
y_train = treinamento['Preco_Dolar']
X_test = teste[['Preco_Dolar_No_Outliers']]
y_test = teste['Preco_Dolar']

# Treinamento do modelo de regressão linear
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Realizando a previsão para os dados de teste
linear_forecast = linear_model.predict(X_test)

# Avaliando o modelo de regressão linear
mse_linear = mean_squared_error(y_test, linear_forecast)
r2_linear = r2_score(y_test, linear_forecast)

# Exibindo os resultados do modelo de regressão linear
print("MSE (Regressão Linear):", mse_linear)
print("R2 Score (Regressão Linear):", r2_linear)

# Preparando os dados de resultado para exibição
result_data_linear = pd.DataFrame({
    'Date': teste['Data'],
    'Real Value': y_test.values,
    'Forecast Value (Regressão Linear)': linear_forecast
})

print(result_data_linear.reset_index(drop=True))


MSE (Regressão Linear): 0.8195936444628833
R2 Score (Regressão Linear): 0.9994403605116554
          Date  Real Value  Forecast Value (Regressão Linear)
0   2022-01-03      249.88                         250.978481
1   2022-01-04      255.79                         256.936704
2   2022-01-05      256.64                         257.793640
3   2022-01-06      257.95                         259.114329
4   2022-01-07      265.18                         266.403322
..         ...         ...                                ...
461 2023-11-06      178.19                         178.703526
462 2023-11-07      178.62                         179.137035
463 2023-11-08      181.48                         182.020371
464 2023-11-09      181.93                         182.474043
465 2023-11-10      180.91                         181.445720

[466 rows x 3 columns]


C:\Users\milen\AppData\Local\Temp\ipykernel_25484\28807278.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Preco_Dolar'] = data['Preco_Dolar'].str.replace('.', '').str.replace(',', '.').astype(float)


In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats

# Carregar o conjunto de dados
file_path = 'C:/Users/milen/OneDrive/Documentos/TCC/new/final.csv' # Substitua pelo caminho correto do seu arquivo
data = pd.read_csv(file_path)

# Convertendo 'Data' para formato datetime e 'Preco_Dolar' para float
# Corrigindo o problema de conversão para float na coluna 'Preco_Dolar'
data['Data'] = pd.to_datetime(data['Data'])
data['Preco_Dolar'] = data['Preco_Dolar'].str.replace('.', '').str.replace(',', '.').astype(float)

# Tratamento de Outliers
z_scores = np.abs(stats.zscore(data['Preco_Dolar']))
threshold = 3
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar'].where(z_scores < threshold, np.nan)

# Interpolando valores NaN resultantes do tratamento de outliers
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar_No_Outliers'].interpolate()


# Separando os dados em conjuntos de treinamento e teste
limite_data = pd.to_datetime("2022-01-01")
treinamento = data[data['Data'] < limite_data]
teste = data[data['Data'] >= limite_data]

X_train = treinamento[['Preco_Dolar_No_Outliers']]
y_train = treinamento['Preco_Dolar']
X_test = teste[['Preco_Dolar_No_Outliers']]
y_test = teste['Preco_Dolar']

# Definindo o tamanho da janela deslizante (por exemplo, 30 dias)
tamanho_janela = 1

# Inicializando listas para armazenar métricas de desempenho
mse_scores = []
r2_scores = []

# Realizando a validação com janela deslizante
for i in range(len(X_test) - tamanho_janela + 1):
    X_train_window = X_train.iloc[i:i + tamanho_janela]
    y_train_window = y_train.iloc[i:i + tamanho_janela]
    X_test_window = X_test.iloc[i:i + tamanho_janela]
    y_test_window = y_test.iloc[i:i + tamanho_janela]

    # Treinamento do modelo Random Forest para esta janela
    rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
    rf_model.fit(X_train_window, y_train_window)

    # Realizando a previsão para os dados desta janela
    rf_forecast = rf_model.predict(X_test_window)

    # Calculando as métricas de desempenho para esta janela
    mse_window = mean_squared_error(y_test_window, rf_forecast)
    r2_window = r2_score(y_test_window, rf_forecast)

    mse_scores.append(mse_window)
    r2_scores.append(r2_window)

# Exibindo as métricas médias de desempenho
print("Média do MSE (Validação com Janela Deslizante):", np.mean(mse_scores))
print("Média do R2 Score (Validação com Janela Deslizante):", np.mean(r2_scores))



C:\Users\milen\AppData\Local\Temp\ipykernel_25484\3459255033.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Preco_Dolar'] = data['Preco_Dolar'].str.replace('.', '').str.replace(',', '.').astype(float)
c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-de

Média do MSE (Validação com Janela Deslizante): 5427.5015517167385
Média do R2 Score (Validação com Janela Deslizante): nan


c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Carregar o conjunto de dados
file_path = 'C:/Users/milen/OneDrive/Documentos/TCC/new/final.csv'  # Substitua pelo caminho correto do seu arquivo
data = pd.read_csv(file_path)

# Convertendo 'Data' para formato datetime e 'Preco_Dolar' para float
data['Data'] = pd.to_datetime(data['Data'])
data['Preco_Dolar'] = data['Preco_Dolar'].str.replace(',', '.').astype(float)

# Dividindo os dados em conjuntos de treinamento, validação e teste
limite_treino = pd.to_datetime("2022-01-01")
limite_validacao = pd.to_datetime("2023-01-01")
dados_treinamento = data[data['Data'] < limite_treino]
dados_validacao = data[(data['Data'] >= limite_treino) & (data['Data'] < limite_validacao)]

# Preparando os conjuntos para treinamento e validação
X_train = dados_treinamento[['Preco_Dolar']]
y_train = dados_treinamento['Preco_Dolar']
X_validacao_1dia = dados_validacao[['Preco_Dolar']].iloc[-1:]  # Último ponto de dados da validação
y_validacao_1dia_real = dados_validacao['Preco_Dolar'].iloc[-1]  # Valor real correspondente

# Treinando o modelo Random Forest com os dados de treinamento
rf_model_1dia = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model_1dia.fit(X_train, y_train)

# Fazendo a previsão para 1 dia usando os dados de validação
previsao_1dia = rf_model_1dia.predict(X_validacao_1dia)

# Exibindo a previsão e o valor real
print("Previsão para 1 dia:", previsao_1dia[0])
print("Valor real:", y_validacao_1dia_real)


Previsão para 1 dia: 196.58939999999984
Valor real: 196.57


In [22]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

# Carregar o conjunto de dados
file_path = 'C:/Users/milen/OneDrive/Documentos/TCC/new/final.csv'  # Substitua pelo caminho correto do seu arquivo
data = pd.read_csv(file_path)

# Convertendo 'Data' para formato datetime e 'Preco_Dolar' para float
data['Data'] = pd.to_datetime(data['Data'])
data['Preco_Dolar'] = data['Preco_Dolar'].str.replace(',', '.').astype(float)

# Dividindo os dados em conjuntos de treinamento, validação e teste
limite_treino = pd.to_datetime("2022-01-01")
limite_validacao = pd.to_datetime("2023-01-01")
dados_treinamento = data[data['Data'] < limite_treino]
dados_validacao = data[(data['Data'] >= limite_treino) & (data['Data'] < limite_validacao)]
dados_teste = data[data['Data'] == data['Data'].max()]

# Preparando os conjuntos para treinamento e validação
X_train = dados_treinamento[['Preco_Dolar']]
y_train = dados_treinamento['Preco_Dolar']
X_validacao = dados_validacao[['Preco_Dolar']]
y_validacao = dados_validacao['Preco_Dolar']

# Treinando o modelo Random Forest com os dados de treinamento
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)

# Validação cruzada com janela deslizante
tscv = TimeSeriesSplit(n_splits=5)
mse_scores = []
r2_scores = []
for train_index, test_index in tscv.split(X_train):
    X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]
    rf_model_cv = RandomForestRegressor(n_estimators=100, random_state=0)
    rf_model_cv.fit(X_train_cv, y_train_cv)
    predictions_cv = rf_model_cv.predict(X_test_cv)
    mse_scores.append(mean_squared_error(y_test_cv, predictions_cv))
    r2_scores.append(r2_score(y_test_cv, predictions_cv))

# Fazendo a previsão para o último dia (teste) usando o último valor da validação
X_teste = [[y_validacao.iloc[-1]]]
y_teste_real = dados_teste['Preco_Dolar'].values[0]
previsao_teste = rf_model.predict(X_teste)

# Exibindo os resultados
print("Previsão para o último dia:", previsao_teste[0])
print("Valor real do último dia:", y_teste_real)
print("MSE médio da validação cruzada:", np.mean(mse_scores))
print("R2 médio da validação cruzada:", np.mean(r2_scores))


Previsão para o último dia: 196.58939999999984
Valor real do último dia: 180.91
MSE médio da validação cruzada: 109.64828581362065
R2 médio da validação cruzada: 0.7803620941562089


c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [23]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Carregar o conjunto de dados
file_path = 'C:/Users/milen/OneDrive/Documentos/TCC/new/final.csv'  # Substitua pelo caminho correto do seu arquivo
data = pd.read_csv(file_path)

# Convertendo 'Data' para formato datetime e 'Preco_Dolar' para float
data['Data'] = pd.to_datetime(data['Data'])
data['Preco_Dolar'] = data['Preco_Dolar'].str.replace(',', '.').astype(float)

# Preparando os dados para o modelo ARIMA
serie_temporal = data['Preco_Dolar']

# Treinando o modelo ARIMA com todos os dados disponíveis, exceto o último dia
arima_model = ARIMA(serie_temporal[:-1], order=(1, 1, 1))
arima_model_fit = arima_model.fit()

# Fazendo a previsão para o último dia
previsao_ultimo_dia_arima = arima_model_fit.forecast(steps=1)
previsao_ultimo_dia_arima_valor = previsao_ultimo_dia_arima.values[0]

# Valor real do último dia
y_ultimo_dia_real = data['Preco_Dolar'].iloc[-1]

# Exibindo a previsão e o valor real
print("Previsão para o último dia (ARIMA):", previsao_ultimo_dia_arima_valor)
print("Valor real do último dia:", y_ultimo_dia_real)


Previsão para o último dia (ARIMA): 181.97007578326046
Valor real do último dia: 180.91
